In [1]:
pip install psycopg2-binary

  Using cached psycopg2-binary-2.9.10.tar.gz (385 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2-binary: filename=psycopg2_binary-2.9.10-cp310-cp310-macosx_11_0_arm64.whl size=133635 sha256=8d9b876ddafee161ba88d5dc891a09c08cef99dbcdaa11e215afa71e43540fa1
  Stored in directory: /Users/bagsolmi/Library/Caches/pip/wheels/bd/8b/58/ff5a880b960d71f645b3679b454f33612254cc39ecdcd6cc5a
Successfully built psycopg2-binary
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


## POSTGRES 데이터베이스 연동하기

In [8]:
import psycopg2
from dotenv import load_dotenv
import os

In [ ]:
# .env 파일 로드
load_dotenv()

# 환경 변수에서 값 가져오기 (DB연결)
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)

cur = conn.cursor() # 쿼리를 실행할 커서 객체 생성
cur.execute("SELECT * FROM country WHERE name = 'United States'")
rows = cur.fetchall()
for row in rows:
    print(row)
# cur.close()

('USA', 'United States', 'North America', 'North America', 9363520.0, 1776, 278357000, 77.1, Decimal('8510700.00'), Decimal('8110900.00'), 'United States', 'Federal Republic', 'George W. Bush', 3813, 'US')


In [11]:
import pandas as pd
pd.read_sql("SELECT * FROM city", conn)

/var/folders/xw/2xv7101s71s4jcv2bxrz47rh0000gn/T/ipykernel_9664/1841372258.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM city", conn)


,id,name,countrycode,district,population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200
...,...,...,...,...,...
4074,4075,Khan Yunis,PSE,Khan Yunis,123175
4075,4076,Hebron,PSE,Hebron,119401
4076,4077,Jabaliya,PSE,North Gaza,113901
4077,4078,Nablus,PSE,Nablus,100231


### 1. SELECT 실습 : 인구가 가장 많은 도시 10개 출력

In [12]:
cr = conn.cursor()

In [15]:
q = """
SELECT name, population
FROM city
ORDER BY population DESC
LIMIT 10
"""
cr.execute(q)
rows = cr.fetchall()
# 결과 출력
for name, population in rows:
    print(f"{name}: {population:,}명")


Mumbai (Bombay): 10,500,000명
Seoul: 9,981,619명
São Paulo: 9,968,485명
Shanghai: 9,696,300명
Jakarta: 9,604,900명
Karachi: 9,269,265명
Istanbul: 8,787,958명
Ciudad de México: 8,591,309명
Moscow: 8,389,200명
New York: 8,008,278명


### 2. INSERT 실습

In [17]:
q = '''INSERT INTO public.city (id, name, countrycode, district, population)
VALUES ({}, '{}', '{}', '{}', {})'''
cr.execute(q.format(99997, 'test city3', 'KOR', 'test 3', 200000))

In [18]:
conn.commit() # 변경사항을 커밋

In [19]:
# 데이터 삽입 후 반영 여부 확인
cr.execute("SELECT * FROM city WHERE id = 99997")
row = cr.fetchone()

if row:
    print("✅ INSERT 성공! 데이터:")
    print(row)
else:
    print("❌ INSERT 실패 또는 커밋 안 됨.")

✅ INSERT 성공! 데이터:
(99997, 'test city3', 'KOR', 'test 3', 200000)


### 3. DELETE 실습

In [20]:
q = "DELETE FROM city WHERE id = %s"
cr.execute(q, (99997,))

conn.commit()  # 삭제 반영

In [21]:
cr.execute("SELECT * FROM city WHERE id = %s", (99997,))
row = cr.fetchone()

if row:
    print("❌ 삭제 실패. 데이터가 아직 존재합니다.")
else:
    print("✅ 삭제 성공! 데이터가 더 이상 존재하지 않습니다.")


✅ 삭제 성공! 데이터가 더 이상 존재하지 않습니다.


## 크롤링 데이터 활용하기

### 크롤링 해 온 yes24 데이터를 book 테이블에 INSERT하기

In [23]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [28]:
from bs4 import BeautifulSoup
import requests

In [29]:
url='https://www.yes24.com/product/category/bestseller?categoryNumber=001001003&pageNumber=1&pageSize=120'
resp = requests.get(url)
# yes_bs = bs4.BeautifulSoup(resp.text)
yes_bs = BeautifulSoup(resp.text, 'html.parser')
div_list = yes_bs.find_all('div', {'class':'item_info'})
book_list = []
for div in div_list:
    temp = {}
    temp['title'] = div.find('a', {'class':'gd_name'}).text
    temp['writer'] = div.find('div', {'class':'info_row info_pubGrp'}).find_all('span')[0].text.strip()
    temp['pub'] = div.find('div', {'class':'info_row info_pubGrp'}).find_all('span')[1].text
    temp['pub_date'] = div.find('div', {'class':'info_row info_pubGrp'}).find_all('span')[2].text
    temp['price'] = div.find('em', {'class':'yes_b'}).text
    if div.find('span', {'class':'rating_grade'}):
        temp['review'] = div.find('span', {'class':'rating_grade'}).find('em').text
    else:
        temp['review'] = None
    book_list.append(temp)

In [30]:
book_list

[{'title': '커서 AI 트렌드&활용백과',
  'writer': '서승완 저',
  'pub': '스마트북스',
  'pub_date': '2025년 03월',
  'price': '22,500',
  'review': '9.6'},
 {'title': '된다! 조회수 터지는 유튜브 쇼츠 만들기',
  'writer': '최지영 저',
  'pub': '이지스퍼블리싱',
  'pub_date': '2025년 04월',
  'price': '19,800',
  'review': '10.0'},
 {'title': '이게 되네? 챗GPT 미친 활용법 71제',
  'writer': '오힘찬 저',
  'pub': '골든래빗',
  'pub_date': '2025년 01월',
  'price': '21,600',
  'review': '9.6'},
 {'title': '진짜 쓰는 실무 엑셀',
  'writer': '전진권 저',
  'pub': '제이펍',
  'pub_date': '2022년 02월',
  'price': '18,900',
  'review': '9.8'},
 {'title': '주니어 백엔드 개발자가 반드시 알아야 할 실무 지식',
  'writer': '최범균 저',
  'pub': '한빛미디어',
  'pub_date': '2025년 04월',
  'price': '25,200',
  'review': None},
 {'title': '비트코인의 시대',
  'writer': '김창익 저',
  'pub': '다산북스',
  'pub_date': '2025년 04월',
  'price': '22,500',
  'review': '10.0'},
 {'title': '요즘 교사를 위한 AI 수업 활용 가이드 with 2022 개정 교육과정',
  'writer': '박진환, 공지훈, 서원진 저',
  'pub': '한빛미디어',
  'pub_date': '2025년 05월',
  'price': '18,000',
  'review': 

In [54]:
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
cur = conn.cursor()

In [55]:
q = '''
      SELECT book_id
      FROM book
      ORDER BY book_id DESC
      LIMIT 1
    '''

cur.execute(q)
last_index = cur.fetchone()[0]

In [ ]:
q = '''
      INSERT INTO book(
      book_id, title, writer, pub, pub_date, review, price)
      VALUES (%s, %s, %s, %s, %s, %s, %s);
    '''

for ix, book in enumerate(book_list, last_index+1):
  cur.execute(q, (ix, book['title'], book['writer'], book['pub'], 
                  book['pub_date'], book['review'], book['price'].replace(',', '')))

conn.commit()  # 변경사항을 커밋


UniqueViolation: duplicate key value violates unique constraint "book_pkey"
DETAIL:  Key (book_id)=(3) already exists.


In [62]:
# 결과 출력
pd.read_sql("SELECT * FROM book", conn)

/var/folders/xw/2xv7101s71s4jcv2bxrz47rh0000gn/T/ipykernel_9664/3209197855.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM book", conn)


,book_id,title,writer,pub,pub_date,review,price
0,1,test book,test writer,test pub,2023-10-01,9.2,15000
1,2,test book,test writer,test pub,2023-10-01,NaN,15000
2,3,커서 AI 트렌드&활용백과,서승완 저,스마트북스,2025년 03월,9.6,22500
3,4,된다! 조회수 터지는 유튜브 쇼츠 만들기,최지영 저,이지스퍼블리싱,2025년 04월,10.0,19800
4,5,이게 되네? 챗GPT 미친 활용법 71제,오힘찬 저,골든래빗,2025년 01월,9.6,21600
...,...,...,...,...,...,...,...
117,118,혼자 공부하는 네트워크,강민철 저,한빛미디어,2024년 04월,9.8,25200
118,119,마인크래프트 미니게임 만들기,우마공 운영진 저,영진닷컴,2024년 02월,9.9,15120
119,120,챗GPT 교사 마스터 플랜,한민철 저,책바세,2023년 10월,9.7,21600
120,121,마인크래프트 핸드북 컬렉션,Mojang Studio 저/이주안 역,영진닷컴,2023년 09월,10.0,37800


In [63]:
cur.close()
cr.close()
conn.close()